This tutorial will teach you how to convert sql to pandas. (And vice versa).
This uses two csv files, which are in this repo:

* buildings.csv [list of tallest building](https://en.wikipedia.org/wiki/List_of_tallest_buildings_in_the_world) taken from wikipedia. 
* continents.csv: A mapping of countries to continents

We also have a buildings.db which is a sqlite file which has the same data as csv files. We will show a sql statement, and then a pandas expression to get the same result. The sql statement can be used with the buildings.db. The required libraries can be installed with `pip install -r requirements.txt`

Let's start by getting the data in a dataframe. `.head` gives us the first 5 rows.

In [37]:
import pandas as pd
df = pd.read_csv("./buildings.csv")
df.head()

,Building,City,Country,Height,Floors,Built
0,Burj Khalifa,Dubai,UAE,828,163,2010
1,Shanghai Tower,Shanghai,China,632,123,2015
2,Abraj Al-Bait Clock Tower,Mecca,Saudi Arabia,601,120,2012
3,Ping An Finance Center,Shenzhen,China,599,115,2015
4,One World Trade Center,New York City,United States,541,104,2014


`.describe` Gives us some info about the dataframe. Not terribly useful, but lets try it.

In [38]:
df.describe()

,Height,Floors,Built
count,121.000000,121.000000,121.000000
mean,358.165289,77.264463,2006.735537
std,79.653034,17.799236,13.691097
min,300.000000,36.000000,1930.000000
25%,308.000000,68.000000,2007.000000
50%,329.000000,75.000000,2011.000000
75%,380.000000,85.000000,2015.000000
max,828.000000,163.000000,2015.000000


We want to get the first 10 results   
`SELECT * FROM buildings LIMIT 10`


In [39]:
df[:10]

,Building,City,Country,Height,Floors,Built
0,Burj Khalifa,Dubai,UAE,828,163,2010
1,Shanghai Tower,Shanghai,China,632,123,2015
2,Abraj Al-Bait Clock Tower,Mecca,Saudi Arabia,601,120,2012
3,Ping An Finance Center,Shenzhen,China,599,115,2015
4,One World Trade Center,New York City,United States,541,104,2014
5,CTF Finance Centre,Guangzhou,China,530,111,2015
6,Taipei 101,Taipei,Republic of China,509,101,2004
7,Shanghai World Financial Center,Shanghai,China,492,101,2008
8,International Commerce Centre The Ritz-Carlton...,Hong Kong,China,484,118,2010
9,Petronas Tower 1,Kuala Lumpur,Malaysia,452,88,1998


Get only the City and Building name   

    SELECT city, building FROM buildings LIMIT 3

In [40]:
 df[["City", "Building"]][:3]

,City,Building
0,Dubai,Burj Khalifa
1,Shanghai,Shanghai Tower
2,Mecca,Abraj Al-Bait Clock Tower


What are the three oldest buildings in our data  

    SELECT city, building, built FROM buildings ORDER BY built LIMIT 3

In [41]:
df.sort("Built")[["City", "Building", "Built"]][:3]

,City,Building,Built
74,New York City,Chrysler Building,1930
29,New York City,Empire State Building,1931
46,Chicago,John Hancock Center,1969


What the the three highest buildings?   

    SELECT city, building, height FROM buildings ORDER BY height desc LIMIT 3

In [42]:
df.sort("Height", ascending=False)[["City", "Building", "Height"]][:3]

,City,Building,Height
0,Dubai,Burj Khalifa,828
1,Shanghai,Shanghai Tower,632
2,Mecca,Abraj Al-Bait Clock Tower,601


What are the three tallest buildings in Dubai?   

    SELECT city, building, height FROM buildings 
    WHERE city='Dubai' ORDER BY height LIMIT 3;
    

In [43]:
df[df["City"]=='Dubai'].sort("Height", ascending=False)[:3]


,Building,City,Country,Height,Floors,Built
0,Burj Khalifa,Dubai,UAE,828,163,2010
15,Marina 101,Dubai,UAE,426,101,2015
19,Princess Tower,Dubai,UAE,414,101,2012


What are the five tallest buildings in Dubai or Chicago?

    SELECT * FROM buildings WHERE city='Dubai' 
    or city=='Chicago' ORDER BY height desc LIMIT 5;

In [44]:
df[(df["City"]=='Dubai') | (df["City"]=="Chicago")].sort("Height", ascending=False)[:5]

,Building,City,Country,Height,Floors,Built
0,Burj Khalifa,Dubai,UAE,828,163,2010
12,Willis Tower (formerly the Sears Tower),Chicago,United States,442,108,1974
15,Marina 101,Dubai,UAE,426,101,2015
17,Trump International Hotel and Towe,Chicago,United States,423,98,2009
19,Princess Tower,Dubai,UAE,414,101,2012


Among the buildings built in 20th century, which has the most floors?

    SELECT * FROM buildings WHERE built < 2000 
    ORDER BY floors DESC LIMIT 1;

In [45]:
df[df["Built"]<2000].sort("Floors", ascending=False)[:1]

,Building,City,Country,Height,Floors,Built
12,Willis Tower (formerly the Sears Tower),Chicago,United States,442,108,1974


Among the buildings built in 20th century, which is the tallest?

    SELECT * FROM buildings WHERE built < 2000
    ORDER BY height DESC LIMIT 1;

In [46]:
df[df["Built"]<2000].sort("Height", ascending=False)[:1]


,Building,City,Country,Height,Floors,Built
9,Petronas Tower 1,Kuala Lumpur,Malaysia,452,88,1998


How many different countries and cities do we have in our data?

    SELECT COUNT(DISTINCT(country)) FROM buildings;
    SELECT COUNT(DISTINCT(city)) FROM buildings;

In [47]:
len(df["Country"].unique())
len(df["City"].unique())

45

Which cities make an apperance in top 20 tallest building list?

    SELECT DISTINCT(city) from 
    (SELECT * FROM buildings ORDER BY height DESC LIMIT 20);
    

In [48]:
 df[:20].sort("Height", ascending=False)["City"].unique()

array(['Dubai', 'Shanghai', 'Mecca', 'Shenzhen', 'New York City',
       'Guangzhou', 'Taipei', 'Hong Kong', 'Kuala Lumpur', 'Nanjing',
       'Chicago'], dtype=object)

How many appearances dos each conutry make in a top 100 list?

    SELECT country, COUNT(country) AS building_count 
    FROM 
    (select * from buildings order by height limit 100) 
    GROUP BY country 
    ORDER BY building_count DESC;
    
    

In [49]:
df[:100]["Country"].value_counts()

China                44
UAE                  24
United States        15
Russia                4
Saudi Arabia          3
Malaysia              3
Republic of China     2
United Kingdom        1
North Korea           1
Australia             1
Kuwait                1
Vietnam               1
dtype: int64

What is the average floors and average height, per country.
   
    SELECT country, AVG(floors), AVG(height) from 
        (SELECT * FROM buildings ORDER BY height DESC LIMIT 100)
        GROUP BY country;

In [50]:
df.sort("Height")[:100].groupby("Country").mean()[["Floors", "Height"]]

,Floors,Height
Country,,
Australia,80.000000,323.000000
Chile,62.000000,300.000000
China,69.886364,330.840909
Japan,60.000000,300.000000
Kuwait,60.000000,300.000000
Malaysia,55.000000,310.000000
North Korea,105.000000,330.000000
Qatar,36.000000,300.000000
Republic of China,85.000000,348.000000


What is the average floors and average height, per country, sorted by height

    SELECT country, AVG(floors) as avg_floor, AVG(height) as avg_height from 
        (SELECT * FROM buildings ORDER BY height DESC LIMIT 100)
        GROUP BY country
        ORDER by avg_height DESC;

In [51]:
df[:100].groupby("Country").mean()[["Floors", "Height"]].sort("Height", ascending=False)

,Floors,Height
Country,,
Saudi Arabia,88.333333,431.333333
Republic of China,93.000000,428.500000
Kuwait,80.000000,413.000000
Malaysia,77.000000,404.666667
China,78.181818,370.886364
UAE,80.583333,365.541667
United States,80.266667,363.466667
Russia,81.750000,344.000000
North Korea,105.000000,330.000000


What years were the buildings completed?

    SELECT built, count(built) FROM buildings GROUP BY built;

In [52]:
df.groupby("Built").count()["Building"]

Built
1930     1
1931     1
1969     1
1973     1
1974     1
1982     1
1983     1
1989     2
1990     2
1992     2
1996     1
1997     3
1998     3
1999     2
2000     2
2001     1
2002     1
2003     1
2004     1
2005     2
2007     4
2008     3
2009     5
2010    11
2011     9
2012    13
2013     6
2014     8
2015    32
Name: Building, dtype: int64

Lets do some joins

    SELECT * from BUILDING LEFT JOIN continents 
    ON buildings.country=continents.country LIMIT 1;

In [53]:
df = pd.read_csv("./buildings.csv")
continents_df = pd.read_csv("./continents.csv")
df.merge(continents_df, on="Country")[:1]

,Building,City,Country,Height,Floors,Built,Unnamed: 0,Continent
0,Burj Khalifa,Dubai,UAE,828,163,2010,1,Asia


What is the average floor count and height on an each continent basis?

    SELECT continent, avg(floors), avg(height) FROM buildings LEFT JOIN continents 
    ON buildings.country=continents.country 
    GROUP by continent;
    

In [54]:
continents_df = pd.read_csv("./continents.csv")
df.merge(continents_df, on="Country").groupby("Continent").mean()[["Floors", "Height"]]

,Floors,Height
Continent,,
Asia,77.010417,361.156250
Australia,80.000000,323.000000
Europe,79.166667,331.166667
North America,78.764706,356.294118
South America,62.000000,300.000000


Thank you. If you *think in sql*, I hope this tutorial allows you to transform your queries to pandas equivalents.